# Average metrics and Analysis using COCO eval API

This .ipynb uses the COCO evaluation API to perform analysis over our models, given our transformed versions of the data sets. Note that this .ipynb can only be run AFTER the 'prepareCocoOutputs.py' script has been run. This notebook performs analysis on a per model, per transform basis.

In [1]:
import os
import glob
import json
import pandas as pd
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import numpy as np
import time

In [7]:
dataDir='../data/inputs'
dataType='val2017'
annFile = '%s/annotations/%s_%s.json'%(dataDir,'instances',dataType)
annType = 'bbox'
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.80s)
creating index...
index created!


### Set up the transform names and model names lists.

In [8]:
#all allowed options
transformNames = [
 'None',
 'gaussianblur_1',
 'gaussianblur_10',
 'gaussianblur_20',
 'superpixels_0p1',
 'superpixels_0p5',
 'superpixels_0p85',
 'colorspace_25',
 'colorspace_50',
 'averageblur_5_11',
 'medianblur_1',
 'sharpen_0',
 'sharpen_1',
 'sharpen_2',
 'addintensity_-80',
 'addintensity_80',
 'elementrandomintensity_1',
 'multiplyintensity_0p25',
 'multiplyintensity_2',
 'contrastnormalization_0',
 'contrastnormalization_1',
 'contrastnormalization_2',
 'elastic_1'
]
modelNames = ['e2e_faster_rcnn_R-50-C4_2x',
'e2e_faster_rcnn_R-50-FPN_2x',
'e2e_faster_rcnn_R-101-FPN_2x',
'e2e_faster_rcnn_X-101-64x4d-FPN_2x',
'e2e_mask_rcnn_R-50-C4_2x',
'e2e_mask_rcnn_R-50-FPN_2x',
'e2e_mask_rcnn_R-101-FPN_2x',
'retinanet_R-50-FPN_2x',
'retinanet_R-101-FPN_2x',
'retinanet_X-101-64x4d-FPN_2x']

### Load the coco formated json output files.

In [41]:
baseOrigPath = '../data/outputs/5000_original_results_coco/'
baseTransformedPath = '../data/outputs/transformed_outputs_coco/'
cocoDt=cocoGt.loadRes(baseTransformedPath + modelNames[3] + '/' + transformNames[3] + '.json')

Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!


### Run the COCOeval evaluation metrics.

In [43]:
imgIds = []
with open(baseTransformedPath + modelNames[0] + '/' + transformNames[3] + '.json', 'r') as f:
    data = json.load(f)
for jsonObject in data:
    imgIds.append(jsonObject['image_id'])
imgIds = list(set(imgIds))

startTime = time.time()
cocoEval = COCOeval(cocoGt,cocoDt,annType)
# Set the evaluation parameters.
cocoEval.params.imgIds = imgIds
cocoEval.params.catIds = [1]
#cocoEval.params.maxDets = [9, 10, 11]
cocoEval.params.iouType = annType
#cocoEval.params.recThrs = [0.1, 0.5, 0.8, 0.9]

# Run the evaluation.
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()
endTime = time.time()
print('Total time to run was %s seconds.'%(str))

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.75s).
Accumulating evaluation results...
DONE (t=0.12s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.027
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.021
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.019
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.034
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.034
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100